In [1]:
import pandas as pd
from datetime import datetime, timedelta


In [2]:

# Load the necessary DataFrames
progress_df = pd.read_csv(r'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/transformed/transformed_progress.csv')
courses_df = pd.read_csv(r'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/transformed/transformed_courses.csv')



In [4]:
fact_courses_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/.venv/mart/dimension/fact_courses.csv'

In [3]:
progress_df['completion_date'] = pd.to_datetime(
    progress_df['updated_year'].astype(str) + '-' +
    progress_df['updated_month'].astype(str) + '-' +
    progress_df['updated_day'].astype(str)
)

In [ ]:
completion_rate = progress_df.groupby('courseId')['percentage_completed'].mean().reset_index()
#completion rate: everage number of days users havve taken to go from 0 to a 100percent completed on a course
courses_df = courses_df.merge(completion_rate, left_on='id', right_on='courseId', how='left')
courses_df.rename(columns={'percentage_completed': 'average_completion_rate'}, inplace=True)

# Step 2: Identify Users at Risk
# Create a function to determine if a user is at risk
def flag_at_risk(row):
    if row['percentage_completed'] < 50:
        course_start_date = progress_df[
            (progress_df['userId'] == row['userId']) & 
            (progress_df['courseId'] == row['courseId']) & 
            (progress_df['percentage_completed'] == 0)
        ]
        if not course_start_date.empty:
            start_day = course_start_date['updated_day'].values[0]
            start_month = course_start_date['updated_month'].values[0]
            start_year = course_start_date['updated_year'].values[0]
            start_date = datetime(year=start_year, month=start_month, day=start_day)

            # Check if 15 days have passed since the course was assigned
            if (datetime.now() - start_date) > timedelta(days=15):
                return 1  # At risk
    return 0  # Not at risk

# Apply the function to determine at risk users
progress_df['at_risk'] = progress_df.apply(flag_at_risk, axis=1)

# Optionally, you can merge this back into courses_df if needed
# courses_df = courses_df.merge(progress_df[['userId', 'courseId', 'at_risk']], on=['userId', 'courseId'], how='left')

# Display results
print(courses_df.head())
print(progress_df.head())


In [ ]:
def calculate_course_completion_rate(transformed_progress_df):
    completion_times = []

    # Group by courseId and calculate completion times
    for course_id, group in transformed_progress_df.groupby('courseId'):
        # Filter for completed courses
        completed_entries = group[group['percentage_completed'] == 100]
        if not completed_entries.empty:
            # Get the start and end dates for completion
            start_dates = group[group['percentage_completed'] == 0][['updated_day', 'updated_month', 'updated_year']]
            end_dates = completed_entries[['updated_day', 'updated_month', 'updated_year']]

            # Ensure there are entries for both start and end
            if not start_dates.empty and not end_dates.empty:
                start_date = datetime(year=start_dates['updated_year'].values[0],
                                      month=start_dates['updated_month'].values[0],
                                      day=start_dates['updated_day'].values[0])
                end_date = datetime(year=end_dates['updated_year'].values[0],
                                    month=end_dates['updated_month'].values[0],
                                    day=end_dates['updated_day'].values[0])
                
                days_to_complete = (end_date - start_date).days
                completion_times.append((course_id, days_to_complete))

    # Create a DataFrame from the completion times
    completion_df = pd.DataFrame(completion_times, columns=['course_id', 'completion_rate'])
    return completion_df

# Calculate course completion rates
course_completion_rate_df = calculate_course_completion_rate(progress_df)

# Merge with dim_courses
dim_courses = fact_courses.merge(course_completion_rate_df, on='course_id', how='left')

# Fill missing values with 0 for completion rates
fact_courses['completion_rate'] = fact_courses['completion_rate'].fillna(0)

# Display the updated dim_courses DataFrame
print("Dim Courses DataFrame with Completion Rates:\n", fact_courses.head())

In [ ]:
# Save the final dimension table to CSV
fact_users_df.to_csv('C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/marts/fact/fact_users.csv', index=False)

print("fact_users.csv created successfully created ")